# EfficientNet CNN

Notebook by Martijn de Vries <br>
martijndevries91@gmail.com

NOTE: the EfficientNetB0 model that I use in this notebook has trouble saving with some version of tensorflow (probably 2.10 and higher) <br>
I used the following fix to make it work on my machine (found in the comments here: https://github.com/keras-team/keras/issues/17199):

location: lib/python3.10/site-packages/keras/applications/efficientnet.py (py3.10) <br>
EDIT this: <br>
<code> x = layers.Rescaling(1.0 / tf.math.sqrt(IMAGENET_STDDEV_RGB))(x) </code> <br>
TO: <br>
<code> x = layers.Rescaling(
    [1.0 / math.sqrt(stddev) for stddev in IMAGENET_STDDEV_RGB]
)(x) </code>



In [38]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

#tensorflow
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.preprocessing.image import img_to_array, load_img, smart_resize
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import regularizers
from tensorflow.keras.callbacks import EarlyStopping

In [39]:
from tensorflow.keras.applications import EfficientNetB0

I can use image_dataset_from_directory to load in the images. How many images are there in each label?

In [40]:
train_ds = image_dataset_from_directory('../processed_images/', image_size=(100,100),
                                      batch_size=200, seed=123, validation_split=0.15,
                                       subset='training', labels ='inferred', color_mode='rgb', label_mode='categorical')  

val_ds = image_dataset_from_directory('../processed_images/', image_size=(100,100),
                                      batch_size=200, seed=123, validation_split=0.15,
                                       subset='validation', labels ='inferred', color_mode='rgb', label_mode='categorical')  

Found 93059 files belonging to 100 classes.
Using 79101 files for training.
Found 93059 files belonging to 100 classes.
Using 13958 files for validation.


In [41]:
#train_ds.class_names

In [42]:
es = EarlyStopping(patience=5)

In [45]:
efficient_net = EfficientNetB0(
    weights='imagenet',
    input_shape=(100, 100, 3),
    include_top=False,
    pooling='max'
)

model = Sequential()
model.add(efficient_net)
model.add(Dropout(0.25))
model.add(Dense(units = 32, activation='relu'))
model.add(Dense(units = 100, activation='softmax'))
model.summary()


model.compile(
    optimizer=Adam(learning_rate = 0.0005),
    loss="categorical_crossentropy",
    metrics=['accuracy']
)

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 efficientnetb0 (Functional)  (None, 1280)             4049571   
                                                                 
 dropout_6 (Dropout)         (None, 1280)              0         
                                                                 
 dense_20 (Dense)            (None, 32)                40992     
                                                                 
 dense_21 (Dense)            (None, 100)               3300      
                                                                 
Total params: 4,093,863
Trainable params: 4,051,840
Non-trainable params: 42,023
_________________________________________________________________


In [46]:
h = model.fit(train_ds, validation_data=val_ds, epochs=30, callbacks=[es])

Epoch 1/30
396/396 [==============================] - 1347s 3s/step - loss: 0.8969 - accuracy: 0.8009 - val_loss: 0.6130 - val_accuracy: 0.8420
Epoch 2/30
396/396 [==============================] - 1285s 3s/step - loss: 0.2663 - accuracy: 0.9242 - val_loss: 0.2543 - val_accuracy: 0.9304
Epoch 3/30
396/396 [==============================] - 1313s 3s/step - loss: 0.1939 - accuracy: 0.9408 - val_loss: 0.2190 - val_accuracy: 0.9423
Epoch 4/30
396/396 [==============================] - 1307s 3s/step - loss: 0.1633 - accuracy: 0.9494 - val_loss: 0.2241 - val_accuracy: 0.9409
Epoch 5/30
396/396 [==============================] - 1306s 3s/step - loss: 0.1493 - accuracy: 0.9522 - val_loss: 0.2331 - val_accuracy: 0.9363
Epoch 6/30
396/396 [==============================] - 1280s 3s/step - loss: 0.1320 - accuracy: 0.9567 - val_loss: 0.2173 - val_accuracy: 0.9428
Epoch 7/30
396/396 [==============================] - 1268s 3s/step - loss: 0.1178 - accuracy: 0.9619 - val_loss: 0.2248 - val_accuracy: